In [8]:
import pandas as pd
import plotly.graph_objects as go
from scipy.cluster.hierarchy import  dendrogram,linkage
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
import numpy as np

In [4]:
import TrajectoryClusteringAnalysis as TCA

In [ ]:
df = pd.read_csv('../data/mvad_data.csv')
# tranformer vos données en format large si c'est n'est pas le cas 
state_mapping = {"EM": 2, "FE": 4, "HE": 6, "JL": 8, "SC": 10, "TR": 12}
colors = ['blue', 'orange', 'green', 'red', 'yellow', 'gray']
df_numeriques = df.replace(state_mapping)
tca = TCA.TCA.TCA(df_numeriques,state_mapping,colors)
tca.bar_treatment_percentage()

In [ ]:
!pip install ../dist/TrajectoryClusteringAnalysis-0.0.3-py3-none-any.whl

In [ ]:
df = pd.read_csv('C:/Users/ndiaga.dieng/Documents/TCA/data/mvad_data.csv')
print(df.head())


In [ ]:
df.columns

In [ ]:
#df_subset = df.iloc[:10, :] 
df_subset = df
df_subset

In [ ]:
drug_mapping = {"EM":2, "FE":4, "HE":6, "JL":8, "SC":10, "TR":12}
df_subsets_numeriques = df_subset.replace(drug_mapping)
df_subsets_numeriques

## TAK (Ndiaga)

In [ ]:
data_array = df_subset.to_numpy()
distance_matrix = pdist(data_array, 'hamming')    
linkage_matrix = linkage(distance_matrix, method='ward',optimal_ordering=True)
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix)
plt.title('Dendrogram of Treatment Sequences')
plt.xlabel('Patients')
plt.ylabel('Distance')
plt.show()

In [7]:
leaves_order= list(hierarchy.leaves_list(linkage_matrix))
data_array = data_array[leaves_order]

In [ ]:
import plotly.graph_objects as go

colors = ['blue', 'orange', 'green', 'red', 'yellow', 'gray']
heatmap = go.Heatmap(z=data_array,colorscale= colors)

# Créer une figure
fig = go.Figure(data=heatmap)
fig.update_layout(
    title='TAK ',
    xaxis_title='Temps',
    yaxis_title='Patients',
)

# Afficher le graphique
fig.show()

# Fitltre modal
Dans l'article original, il est mentionné qu'un filtre modal est utilisé pour cette étape

In [11]:
import numpy as np

def modal_filter(data, kernel_size):
    filtered_data = np.full_like(data, np.nan)
    kernel_rows, kernel_cols = kernel_size
    
    for i in range(len(data)):
        row_min = max(0, i - kernel_rows // 2)
        row_max = min(len(data), i + kernel_rows // 2 + 1)
        for j in range(len(data[i])):
            col_min = max(0, j - kernel_cols // 2)
            col_max = min(len(data[i]), j + kernel_cols // 2 + 1)
            kernel_values = data[row_min:row_max, col_min:col_max].flatten()
            # Filter out negative values and NaN
            kernel_values = kernel_values[~np.isnan(kernel_values)]
            if len(kernel_values) > 0:
                # Convert kernel_values to integers
                kernel_values_int = kernel_values.astype(int)
                most_frequent_value = np.argmax(np.bincount(kernel_values_int))
                filtered_data[i, j] = most_frequent_value
    
    return filtered_data



filtered_data_opt = modal_filter(data_array, (10, 7))

In [ ]:


heatmap1 = go.Heatmap(z=filtered_data_opt,colorscale= colors)

# Créer une figure
fig = go.Figure(data=heatmap1)
fig.update_layout(
    title='Ordered Treatment Sequences  optimal leaves',
    xaxis_title='Temps',
    yaxis_title='Patients',
)

# Afficher le graphique
fig.show()

In [ ]:

# List of treatments
treatments = ["EM", "FE", "HE", "JL", "SC", "TR"]

# Define colors for treatments
colors = ['blue', 'orange', 'green', 'red', 'yellow', 'gray']

# Create a figure
fig = go.Figure()

# Iterate over treatments
for treatment, color in zip(treatments, colors):
    # Extract data for the current treatment
    treatment_data = df[df.eq(treatment).any(axis=1)]
    # Get the columns representing months
    months = treatment_data.columns
    # Calculate the percentage of patients under the current treatment for each month
    percentages = (treatment_data.apply(lambda x: x.value_counts().get(treatment, 0)) / len(treatment_data)) * 100
    # Plot the curve for the percentage of patients under the treatment over time
    fig.add_trace(go.Scatter(x=months, y=percentages, mode='lines', name=treatment, line=dict(color=color)))

# Update layout
fig.update_layout(
    title='Percentage of Patients under Each Treatment Over Time',
    xaxis_title='Time',
    yaxis_title='Percentage of Patients',
    legend_title='Treatment',
    yaxis=dict(tickformat=".2f")
)

# Display the graph
fig.show()
